In [2]:
#also requires the following modules to be loaded: py-jupyter/1.0.0_py36, python/3.6.1, py-ants/0.3.2_py36

import os
import numpy as np
import nibabel as nib
import ants
from shutil import copyfile

In [3]:
dir = '/oak/stanford/groups/trc/data/Jacob/ImagingData/Bruker/20231117/moco/'
files = os.listdir(dir)
files = [f for f in files if f.endswith('_moco.nii')]

# import mean image from moco nii stacks
# load mean images
image1np = np.mean(np.asarray(nib.load(dir + files[0]).get_data(),dtype='float32'), axis=3)
image2np = np.mean(np.asarray(nib.load(dir + files[1]).get_data(),dtype='float32'), axis=3)

image1 = ants.from_numpy(image1np)
image2 = ants.from_numpy(image2np)

grad_step = 0.2
flow_sigma = 3
total_sigma = 0
syn_sampling = 32
type_of_transform = 'Translation'

moco = ants.registration(image1,image2,type_of_transform=type_of_transform,grad_step=grad_step, flow_sigma=flow_sigma,total_sigma=total_sigma)
    #syn_sampling=syn_sampling)

fwdtransforms = moco['fwdtransforms']
print(fwdtransforms)


/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning:

get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0

/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0



['/tmp/tmp0n2r68vk0GenericAffine.mat']


In [4]:
#save transform
fwdtransforms_save_dir = os.path.join(dir, 'fwdtransforms')
if not os.path.exists(fwdtransforms_save_dir):
    os.mkdir(fwdtransforms_save_dir)
for source_path in fwdtransforms:
    source_file = source_path.split('/')[-1]
    target_path = os.path.join(fwdtransforms_save_dir, source_file)
    copyfile(source_path, target_path)


In [5]:
#load and print transform
tx = ants.read_transform(target_path)
tx_params = np.asarray(tx.parameters)
tx_params

array([1.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 1.        , 0.95388794,
       5.14637756, 2.66687012])

In [6]:
# register shifted image

#add 5 frames in z dim
image1npshift = np.zeros((image1np.shape[0], image1np.shape[1], image1np.shape[2]+5))
image2npshift = np.zeros((image2np.shape[0], image2np.shape[1], image2np.shape[2]+5))
#shift images
image1npshift[:,:,0:image1np.shape[2]] = image1np
image2npshift[:,:,5:image2np.shape[2]+5] = image2np
#convert to ants
image1shift = ants.from_numpy(image1npshift)
image2shift = ants.from_numpy(image2npshift)
grad_step = 0.2

moco_shift = ants.registration(image1shift,image2shift,type_of_transform=type_of_transform,grad_step=grad_step, flow_sigma=flow_sigma,total_sigma=total_sigma)
    #syn_sampling=syn_sampling)

fwdtransforms_shift = moco_shift['fwdtransforms']
print(fwdtransforms_shift)

['/tmp/tmp5d94zrtx0GenericAffine.mat']


In [7]:
#save transform
for source_path in fwdtransforms_shift:
    source_file = source_path.split('/')[-1]
    target_path = os.path.join(fwdtransforms_save_dir, source_file)
    copyfile(source_path, target_path)


In [8]:
#load transform and figure out which is z dim
tx_shift = ants.read_transform(target_path)
txshift_params = np.asarray(tx_shift.parameters)
tx_params - txshift_params #number ~5 indicates which dim corresponds to shift in z: turns out its x y z shift as the last 3 args in the paramaeter array

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.14440918e-04, -4.08935547e-03, -5.01094913e+00])

In [9]:
#final result

#scan resolution microns/voxel
x_res=0.217597749403822
y_res=0.217597749403822
z_res=0.5

#calculate shift in microns
x_shift = tx_params[9]*x_res
y_shift = tx_params[10]*y_res
z_shift = tx_params[11]*z_res
print('x,y,z shift in microns between 920nm and 1064nm excitation is ' + np.array2string(x_shift) + ', ' + np.array2string(y_shift) + ', ' + np.array2string(z_shift))

x,y,z shift in microns between 920nm and 1064nm excitation is 0.20756387, 1.11984018, 1.33343506
